In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import spacy
from spacy import displacy

In [2]:
url = "https://www.nbr.org/publication/u-s-australia-energy-cooperation-in-the-indo-pacific/"
response = requests.get(url)

In [3]:
soup=BeautifulSoup(response.content, "html.parser")

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   U.S.-Australia Energy Cooperation in the Indo-Pacific | The National Bureau of Asian Research (NBR)
  </title>
  <link href="https://www.nbr.org/xmlrpc.php" rel="pingback"/>
  <link href="https://www.nbr.org/wp-content/themes/nbr-theme/build/css/main.css" rel="stylesheet" type="text/css"/>
  <link href="https://www.nbr.org/wp-content/themes/nbr-theme/style.css" media="screen" rel="stylesheet" type="text/css"/>
  <script crossorigin="anonymous" defer="" integrity="sha384-3yBLeJ4waqGSAf4A8pjZ13UF7GuhgbdKnBQvIp/TkWoXtQbtwjlIPNjkDRJ46UCn" src="https://pro.fontawesome.com/releases/v5.5.0/js/all.js">
  </script>
  <meta content="max-image-preview:large" name="robots"/>
  <link href="//code.jquery.com" rel="dns-prefetch"/>
  <s

In [5]:
main_content = soup.find('div',{'class':'publication-content'})
for element in main_content(['a', 'em']):
    element.decompose()

In [6]:
text_list = []
stop_text = "She also serves as the official U.S. delegate to the Energy Research Institute Network, an East Asia Summit–linked network whose inputs are designed to inform the formal East Asia Summit process."
for paragraph in main_content.find_all('p'):
    paragraph_text = paragraph.get_text(separator=' ', strip=False)

    if stop_text in paragraph_text:
        text_list.append(paragraph_text)
        break
    text_list.append(paragraph_text)

In [7]:
df = pd.DataFrame(text_list, columns=["Paragraph Text"])
df.to_csv('09_paragraphs.csv', index=False)

In [8]:
df

,Paragraph Text
0,Can fierce competitors make for the best of al...
1,Both countries have substantial natural resour...
2,"Despite these competing interests, the United ..."
3,An important cornerstone of any collaboration ...
4,In February 2018 the Trump and Morrison admini...
5,The United States currently has only three str...
6,Open-ended flexibility is the hallmark of a su...
7,According to estimates by the U.S. Energy Info...
8,"Recognizing this, the U.S.-Australia Strategic..."
9,"Meanwhile, the second identified area of coope..."


In [9]:
nlp = spacy.load("en_core_web_sm")

In [10]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [14]:
relationship_keywords = {'collaboration', 'cooperation', 'partner', 'engage', 
                         'support', 'fund', 'finance', 'invest', 'develop', 
                         'supply', 'promote', 'transition', 'export', 'agreement', 
                         'interest', 'share', 'enhance', 'effort', 'import'}

In [15]:
def extract_entities_and_relationships(text):
    doc = nlp(text)  # Process text with spaCy
    entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['ORG', 'GPE', 'FAC']]

    # Find relationships based on keywords
    relationships = []
    for token in doc:
        if token.lemma_ in relationship_keywords:  # Use lemma_ to check relationship keywords
            subject = [w for w in token.children if w.dep_ == "nsubj"]  # Find subject
            object_ = [w for w in token.children if w.dep_ == "dobj"]  # Find object
            if subject and object_:
                relationships.append((subject[0].text, token.text, object_[0].text))  # (subject, relationship, object)

    return entities, relationships

In [16]:
df['Entities'], df['Relationships'] = zip(*df['Paragraph Text'].apply(extract_entities_and_relationships))

In [17]:
df[['Paragraph Text', 'Entities', 'Relationships']]

,Paragraph Text,Entities,Relationships
0,Can fierce competitors make for the best of al...,"[(the United States, GPE), (Australia, GPE)]",[]
1,Both countries have substantial natural resour...,"[(U.S., GPE), (the Indo-Pacific, ORG), (The In...",[]
2,"Despite these competing interests, the United ...","[(the United States, GPE), (Australia, GPE), (...",[]
3,An important cornerstone of any collaboration ...,"[(the United States, GPE), (Australia, GPE), (...",[]
4,In February 2018 the Trump and Morrison admini...,"[(Trump, ORG), (Australia, GPE)]",[]
5,The United States currently has only three str...,"[(The United States, GPE), (the Indo-Pacific, ...",[]
6,Open-ended flexibility is the hallmark of a su...,"[(the United States, GPE), (Australia, GPE)]",[]
7,According to estimates by the U.S. Energy Info...,"[(the U.S. Energy Information Administration, ...",[]
8,"Recognizing this, the U.S.-Australia Strategic...",[(the U.S.-Australia Strategic Energy Partners...,[]
9,"Meanwhile, the second identified area of coope...","[(LNG, ORG), (the United States, GPE), (Austra...",[]


In [18]:
for index, row in df.iterrows():
    print(f"\nParagraph {index + 1}:")
    print(row['Paragraph Text'])

    # Display dependency parsing
    doc = nlp(row['Paragraph Text'])
    sentence_spans = list(doc.sents)
    displacy.render(sentence_spans, style='dep', jupyter=True, options={'compact': True})
    
    # Display named entities
    displacy.render(sentence_spans, style="ent", jupyter=True)
    
    # Print extracted entities and relationships
    print("Entities:", row['Entities'])
    print("Relationships:", row['Relationships'])


Paragraph 1:
Can fierce competitors make for the best of allies? If the two powers involved are the United States and Australia, that answer is seemingly yes.


/Users/loogyee/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Entities: [('the United States', 'GPE'), ('Australia', 'GPE')]
Relationships: []

Paragraph 2:
Both countries have substantial natural resource endowments (particularly in coal and natural gas) and significant competitive strengths in the export of these resources as well as technologies associated with solar, wind, and more efficient power generation. Government officials and industry leaders in both countries also regularly tout trade in energy and energy technologies as a critical driver of their respective country’s economic potential—and the market that U.S. and Australian exporters are most aggressively pursuing is one and the same: the Indo-Pacific. The Institute Of Energy Economics, Japan, now projects that by 2050 80% of all energy traded globally will be consumed in Asia.  This makes the Indo-Pacific the essential region for exporters in both countries.


Entities: [('U.S.', 'GPE'), ('the Indo-Pacific', 'ORG'), ('The Institute Of Energy Economics', 'ORG'), ('Japan', 'GPE'), ('the Indo-Pacific', 'ORG')]
Relationships: []

Paragraph 3:
Despite these competing interests, the United States and Australia have maintained a close partnership on regional energy cooperation. This brief explores the dynamics of this partnership and assesses the overall outlook for future cooperation in the Indo-Pacific. Specifically, it highlights areas in which the United States and Australia are already working together, as well as those where renewed or expanded cooperation could play an important role in increasing regional economic prosperity.


Entities: [('the United States', 'GPE'), ('Australia', 'GPE'), ('the Indo-Pacific', 'ORG'), ('the United States', 'GPE'), ('Australia', 'GPE')]
Relationships: []

Paragraph 4:
An important cornerstone of any collaboration is shared values. In broad terms, both the United States and Australia follow a market-based approach to energy policymaking that is backed by liberal democratic values. Both countries generally understand that the government’s role in the energy sector is to deploy a range of tools (e.g., regulations, standards, and investments) to nudge sectorial decisions into alignment with high-level goals such as promoting economic growth, combating air pollution, or guaranteeing labor safeguards.  Historically, this has meant resisting the urge to deploy resource advantages as a geopolitical weapon, lest it undermine confidence in international trade. When this approach is coupled with broader commitments to fostering open markets, the result to date has been an environment whe

...

Entities: [('the United States', 'GPE'), ('Australia', 'GPE'), ('the United States', 'GPE'), ('the U.S. International Development Finance Corporation', 'ORG'), ('the Better Utilization of Investments Leading to Development (BUILD) Act', 'ORG'), ('U.S.', 'GPE')]
Relationships: []

Paragraph 16:
Both the United States and Australia already play an important role in reducing energy insecurities in Northeast Asia, and they continue to expand their ambitions in Southeast Asia, South Asia, and the Pacific. Yet at the same time, neither country could meet all of Asia’s projected energy demands on its own (or even in tandem). As noted by the head of the IEA, just meeting Asia’s rising demand for natural gas would require the United States to build one new LNG export terminal every five months.


Entities: [('the United States', 'GPE'), ('Australia', 'GPE'), ('IEA', 'ORG'), ('the United States', 'GPE'), ('LNG', 'ORG')]
Relationships: []

Paragraph 17:
Consequently, even as both countries look at how they might directly meet Asia’s energy requirements, this cannot replace the broader requirement for healthy, well-functioning regional and global markets. And in turn, fostering such markets will require dedicated, long-term support for sustained reform efforts. These include ongoing initiatives to liberalize markets, discourage the politicization of energy, and strengthen regional energy architecture to reduce any one country’s exposure to risks stemming from price or supply shocks. And there is only so much of this that can be done in the bilateral, or even trilateral, realm.


Entities: []
Relationships: []

Paragraph 18:
Good alliances are adaptive, flexible, and contain breathing room that allows them to think creatively about how to best prepare for contingencies. The United States and Australia already have such an alliance on energy cooperation. But taking this to new heights as part of a strategy truly focused on the Indo-Pacific will require greater commitment.


Entities: [('The United States', 'GPE'), ('Australia', 'GPE'), ('the Indo-Pacific', 'ORG')]
Relationships: []

Paragraph 19:
Clara Gillispie is a CFR International Affairs Fellow at Carnegie India and Senior Advisor to the National Bureau of Asian Research. She also serves as the official U.S. delegate to the Energy Research Institute Network, an East Asia Summit–linked network whose inputs are designed to inform the formal East Asia Summit process.


Entities: [('CFR International Affairs Fellow', 'ORG'), ('Carnegie India', 'ORG'), ('the National Bureau of Asian Research', 'ORG'), ('U.S.', 'GPE'), ('the Energy Research Institute Network', 'ORG')]
Relationships: []


In [19]:
entity1_list = []
relationship_list = []
entity2_list = []
date_list = []
money_list = []

In [20]:
# Iterate through each paragraph
for index, row in df.iterrows():
    doc = nlp(row['Paragraph Text'])  # Process the paragraph with spaCy

    for sent in doc.sents:
        # Extract named entities (countries, organizations)
        entities = []
        money_in_sentence = []

        for ent in sent.ents:
            if 'Inititative' in ent.text.lower(): 
                ent = ent._replace(label_='POL')
                entities.append(ent)
            elif ent.label_ in ['GPE', 'ORG', 'POL', 'MONEY']:
                entities.append(ent)

            # Collect monetary amounts
            if ent.label_ == "MONEY":
                money_in_sentence.append(ent.text)
        
        # Look for verbs or specific relationship nouns in the sentence
        verbs = [token for token in sent if token.pos_ == "VERB"]
        keywords_in_sentence = [token.text for token in sent if token.lemma_ in relationship_keywords]
        
        # Extract dates
        dates = [ent.text for ent in sent.ents if ent.label_ == "DATE"]

        # If we find two entities and a relationship keyword, we assume a relationship
        if len(entities) >= 2 and keywords_in_sentence:
            entity1_list.append(entities[0].text)

            # Concatenate the relationship noun and verb (if available) into one string
            relationship_info = keywords_in_sentence[0]
            if verbs:
                relationship_info += " (verb: " + verbs[0].lemma_ + ")"

            relationship_list.append(relationship_info)  # Store concatenated info
            entity2_list.append(entities[1].text)
            date_list.append(dates[0] if dates else "n/a")
            money_list.append(money_in_sentence[0] if money_in_sentence else "n/a")

In [21]:
# DataFrame
relationships_df = pd.DataFrame({
    "Entity1": entity1_list,
    "Relationship": relationship_list,
    "Entity2": entity2_list,
    "Date": date_list,
    "Money": money_list
})

In [22]:
relationships_df

,Entity1,Relationship,Entity2,Date,Money
0,the United States,interests (verb: compete),Australia,n/a,n/a
1,the United States,cooperation (verb: highlight),Australia,n/a,n/a
2,U.S.,engage (verb: couple),Australia,n/a,n/a
3,India,collaboration (verb: include),Japan,n/a,n/a
4,the U.S.-Australia Strategic Energy Partnership,partner (verb: recognize),the United States,n/a,n/a
5,LNG,exports (verb: liquefy),the United States,n/a,n/a
6,The Asian Development Bank,developing (verb: estimate),$26 trillion,between 2016 and 2030,$26 trillion
7,more than $1 billion,partners (verb: announce),LNG,June 2019,more than $1 billion
8,the United States,support (verb: look),the U.S. International Development Finance Cor...,n/a,n/a
9,IEA,export (verb: note),the United States,every five months,n/a


In [23]:
# Export to csv
relationships_df.to_csv('09_relationships.csv', index=False)